In [ ]:
__author__ = 'Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240131' # yyyymmdd; version datestamp of this notebook
#__datasets__ = ['allwise','smash']
__keywords__ = ['gsaoi','gemini','galaxy','dragons']

# Gemini GSAOI reduction using DRAGONS Python API
***
## Public archival data from gsaoiimg_tutorial - GS-2017A-Q-29 (NGC5128)
#### adapted from https://dragons.readthedocs.io/projects/gsaoiimg-drtutorial/en/v3.1.0/index.html
#### don't forget to `conda install -n dragons nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env
***

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create Master Flat Field](#Master_Flat)
* [Create File Lists](#File_Lists)
* [Standard Star](#Standard_Star)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)


<a class="anchor" id="goals"></a>
# Goals
Showcase how to perform GSAOI imaging data reduction using the Gemini DRAGONS package on the Data Lab science platform. Uses a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing of flats, darks, bad pixel mask, and science frames, and finally the creation of a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It currently can be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, and spectroscopic data in GMOS longslit mode. Linked here is a general list of guides, manuals, and tutorials about the use of DRAGONS:
https://dragons.readthedocs.io/en/v3.1.0/

The DRAGONS kernel has been made available in the Data Lab environment, which should allow users to access the routines without being dependent on installing the software in their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to fully reduce example Gemini South NIRI Kshort-band imaging data.
This is a version of the DRAGONS Jupyter notebook tutorial presented here: 
https://dragons.readthedocs.io/projects/gsaoiimg-drtutorial/en/v3.1.0/ex1_gsaoiim_offsetsky_api.html

This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS GSAOI data reduction process, rather will just show one example of a standard reduction of a GSAOI imaging dataset. 
More extensive explanations can be found in the general DRAGONS GNIRS data reduction tutorial from Gemini linked here:
https://dragons.readthedocs.io/projects/gsaoiimg-drtutorial/en/v3.1.0/index.html

The data used in this notebook example is GSAOI KshortH-band imaging from the Gemini archive of the giant elliptical galaxy NGC 5128 from the Gemini South program "Stellar Population GeMology: Long Period Variables at High Metallicity in the Nearest Elliptical Galaxy", PI: John Blakeslee, program ID GS-2017A-Q-29. More program information is given here: https://archive.gemini.edu/programinfo/GS-2017A-Q-29.

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* DRAGONS publication: Labrie et al., "DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South", ASPC, 523, 321L, https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract

* DRAGONS open source software publication: https://zenodo.org/record/7776065#.ZDg5qOzMLUI


**Importing Python Libraries** (you'll probably have to install the `wget`, `disco_stu`, and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

#DRAGONS
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.adlibrary import dataselect
from gempy.utils import logutils

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

from collections import namedtuple

#%matplotlib widget
%matplotlib inline

<a class="anchor" id="About"></a>
# About the dataset

| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | S20170505S0095-110 | Kshort-band, on target, 60 s |
| Flats | S20170505S0030-044 | Lamp on, Kshort, for science |
| Flats | S20170505S0060-074 | Lamp off, Kshort, for science |
| Science darks | S20170504S0114-117 | Kshort, standard star, 30 s |

Note that for GSAOI, the dark current is low enough that there is no need to correct for it.

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data (direct link to .tar file hosted at Gemini - 960 Megabytes)

In [ ]:
#wget.download("http://www.gemini.edu/sciops/data/software/datapkgs/gsaoiimg_tutorial_datapkg-v1.tar")

**Downloading the data** (individual files - direct link to the Gemini Archive - 3.0 Gigabytes)

In [ ]:
# # uncomment the lines and run the cell

## Science 
#
#wget.download("http://archive.gemini.edu/file/S20170505S0095.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0096.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0097.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0098.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0099.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0100.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0101.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0102.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0103.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0104.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0105.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0106.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0107.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0108.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0109.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0110.fits")
#
## Flats 
#
#wget.download("http://archive.gemini.edu/file/S20170505S0030.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0031.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0032.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0033.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0034.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0035.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0036.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0037.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0038.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0039.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0040.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0041.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0042.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0043.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0044.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0060.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0061.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0062.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0063.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0064.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0065.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0066.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0067.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0068.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0069.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0070.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0071.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0072.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0073.fits")
#wget.download("http://archive.gemini.edu/file/S20170505S0074.fits")
#
## Standard star
#
#wget.download("http://archive.gemini.edu/file/S20170504S0114.fits")
#wget.download("http://archive.gemini.edu/file/S20170504S0115.fits")
#wget.download("http://archive.gemini.edu/file/S20170504S0116.fits")
#wget.download("http://archive.gemini.edu/file/S20170504S0117.fits")

**Create and move data to raw/ directory** (uncomment first)

In [ ]:
#!mkdir raw/
#!mv S2017*.fits raw/

In [ ]:
# Check header of one raw science image
#
tmp = fits.open("raw/S20170505S0095.fits")
tmp[0].header

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This simply tells the system where to put the calibration database, the database that will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='gsaoi_data_reduction.log')

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('raw/S2017*[0-9].fits')
all_files.sort()
#all_files

<a class="anchor" id="File_Lists"></a>
# Create file lists

The first step is to create input file lists. The tool “dataselect” helps with that. It uses Astrodata tags and “descriptors” to select the files and send the filenames to a text file that can then be fed to “reduce”. (See the Astrodata User Manual for information about Astrodata.)

In [ ]:
list_of_flats_Ks = dataselect.select_data(
     all_files,['FLAT'],[],
     dataselect.expr_parser('filter_name=="Kshort"'))

list_of_std_stars = dataselect.select_data(
    all_files,[],[],
    dataselect.expr_parser('observation_class=="partnerCal"'))

list_of_science_images = dataselect.select_data(
    all_files,[],[],
    dataselect.expr_parser('(observation_class=="science" and exposure_time==60.)'))

<a class="anchor" id="Master_Flat"></a>
# Create Master Flat Field

The GSAOI Kshort master flat is created from a series of lamp-on and lamp-off dome exposures. They should all have the same exposure time. Each flavor is stacked (averaged), then the lamp-off stack is subtracted from the lamp-on stack and the result normalized.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats_Ks)
reduce_flats.runr()

<a class="anchor" id="Standard_Star"></a>
# Reduce Standard Star

The standard star is reduced essentially the same way as the science target (next section). The processed flat field that we added earlier to the local calibration database will be fetched automatically. Also, in this case the standard star was obtained using ROIs (Regions-of-Interest) which do not match the flat field. The software will recognize that the flat field is still valid and will crop it to match the ROIs.

In [ ]:
reduce_std = Reduce()
reduce_std.files.extend(list_of_std_stars)
reduce_std.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce Science Images

This is an observation of a galaxy with offset to sky. We need to turn off the additive offsetting of the sky because the target fills the field of view and does not represent a reasonable sky background. If the offsetting is not turned off in this particular case, it results in an over-subtraction of the sky frame.

**Note**:
Unlike the other near-IR instruments, the additive offset_sky parameter is used by default to adjust the sky frame background for GSAOI instead of the multiplicative scale_sky parameter. It was found to work better when the sky background per pixel is very low, which is common due to the short exposure time needed to avoid saturating stars and the small pixel scale. The reader is encourage to experiment with scale_sky if offset_sky does not seem to lead to an optimal sky subtraction.

(Remember that when the source is extended, both parameters normally need to be turned off.)

The sky frame comes from off-target sky observations. We feed the pipeline all the on-target and off-target frames. The software will split the on-target and the off-target appropriately using information in the headers.

This command will generate flat corrected and sky subtracted files but will not stack them. You can find which file is which by its suffix (_flatCorrected or _skyCorrected). The on-target files are the ones that have been sky subtracted (_skyCorrected). There should be nine of them.

In [ ]:
reduce_target = Reduce()
reduce_target.files.extend(list_of_science_images)
reduce_target.uparms.append(('skyCorrect:offset_sky', False))
reduce_target.runr()

<a class="anchor" id="Display_Image"></a>
# Display the Stacked Image

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the “SCI” extension, the variance is in the “VAR” extension, and the data quality plane (mask) is in the “DQ” extension.

In [ ]:
image_file = 'S20170505S0095_image.fits'
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (15,15))
plt.imshow(image_data,cmap='bone',norm=Normalize(vmin=1, vmax=1000),origin='lower')
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove duplicate calibrations and remove raw data (uncomment lines before running)

In [ ]:
#!rm -rf *_flatCorrected.fits *_skySubtracted*.fits
#!rm -rf raw/